<a href="https://colab.research.google.com/github/sarahxu087/DS-Unit-1-Sprint-1-Data-Wrangling-and-Storytelling/blob/master/module3-join-and-reshape-data/Copy_of_LS_DS10_113_Join_and_Reshape_Data_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 1, Sprint 1, Module 3*

---

# Join and Reshape datasets

Objectives
- concatenate data with pandas
- merge data with pandas
-  understand tidy data formatting
-  melt and pivot data with pandas

Links
- [Pandas Cheat Sheet](https://github.com/pandas-dev/pandas/blob/master/doc/cheatsheet/Pandas_Cheat_Sheet.pdf)
- [Tidy Data](https://en.wikipedia.org/wiki/Tidy_data)
  - Combine Data Sets: Standard Joins
  - Tidy Data
  - Reshaping Data
- Python Data Science Handbook
  - [Chapter 3.6](https://jakevdp.github.io/PythonDataScienceHandbook/03.06-concat-and-append.html), Combining Datasets: Concat and Append
  - [Chapter 3.7](https://jakevdp.github.io/PythonDataScienceHandbook/03.07-merge-and-join.html), Combining Datasets: Merge and Join
  - [Chapter 3.8](https://jakevdp.github.io/PythonDataScienceHandbook/03.08-aggregation-and-grouping.html), Aggregation and Grouping
  - [Chapter 3.9](https://jakevdp.github.io/PythonDataScienceHandbook/03.09-pivot-tables.html), Pivot Tables
  
Reference
- Pandas Documentation: [Reshaping and Pivot Tables](https://pandas.pydata.org/pandas-docs/stable/reshaping.html)
- Modern Pandas, Part 5: [Tidy Data](https://tomaugspurger.github.io/modern-5-tidy.html)

In [1]:
!wget https://s3.amazonaws.com/instacart-datasets/instacart_online_grocery_shopping_2017_05_01.tar.gz

--2019-10-31 01:06:20--  https://s3.amazonaws.com/instacart-datasets/instacart_online_grocery_shopping_2017_05_01.tar.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.84.141
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.84.141|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 205548478 (196M) [application/x-gzip]
Saving to: ‘instacart_online_grocery_shopping_2017_05_01.tar.gz’

instacart_online_gr 100%[===================>] 196.03M  40.1MB/s    in 4.6s    

2019-10-31 01:06:25 (42.3 MB/s) - ‘instacart_online_grocery_shopping_2017_05_01.tar.gz’ saved [205548478/205548478]



In [2]:
!tar --gunzip --extract --verbose --file=instacart_online_grocery_shopping_2017_05_01.tar.gz

instacart_2017_05_01/
instacart_2017_05_01/._aisles.csv
instacart_2017_05_01/aisles.csv
instacart_2017_05_01/._departments.csv
instacart_2017_05_01/departments.csv
instacart_2017_05_01/._order_products__prior.csv
instacart_2017_05_01/order_products__prior.csv
instacart_2017_05_01/._order_products__train.csv
instacart_2017_05_01/order_products__train.csv
instacart_2017_05_01/._orders.csv
instacart_2017_05_01/orders.csv
instacart_2017_05_01/._products.csv
instacart_2017_05_01/products.csv


In [3]:
%cd instacart_2017_05_01

/content/instacart_2017_05_01


In [4]:
!ls -lh *.csv

-rw-r--r-- 1 502 staff 2.6K May  2  2017 aisles.csv
-rw-r--r-- 1 502 staff  270 May  2  2017 departments.csv
-rw-r--r-- 1 502 staff 551M May  2  2017 order_products__prior.csv
-rw-r--r-- 1 502 staff  24M May  2  2017 order_products__train.csv
-rw-r--r-- 1 502 staff 104M May  2  2017 orders.csv
-rw-r--r-- 1 502 staff 2.1M May  2  2017 products.csv


# Assignment

## Join Data Practice

These are the top 10 most frequently ordered products. How many times was each ordered? 

1. Banana
2. Bag of Organic Bananas
3. Organic Strawberries
4. Organic Baby Spinach 
5. Organic Hass Avocado
6. Organic Avocado
7. Large Lemon 
8. Strawberries
9. Limes 
10. Organic Whole Milk

First, write down which columns you need and which dataframes have them.

Next, merge these into a single dataframe.

Then, use pandas functions from the previous lesson to get the counts of the top 10 most frequently ordered products.

In [5]:
##### YOUR CODE HERE #####
import pandas as pd

order_products__prior = pd.read_csv('order_products__prior.csv')
print(order_products__prior.shape)
order_products__prior.head()

(32434489, 4)


,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


In [6]:
order_products__train = pd.read_csv('order_products__train.csv')
print(order_products__train.shape)
order_products__train.head()

(1384617, 4)


,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1
2,1,10246,3,0
3,1,49683,4,0
4,1,43633,5,1


In [7]:
orders = pd.read_csv('orders.csv')
print(orders.shape)
orders.head()

(3421083, 7)


,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0


In [8]:
condition = (orders['user_id'] == 1) & (orders['order_number'] <=2)

columns = ['user_id', 
           'order_id', 
           'order_number', 
           'order_dow', 
           'order_hour_of_day']

orders_subset = orders.loc[condition, columns]
orders_subset.head()

,user_id,order_id,order_number,order_dow,order_hour_of_day
0,1,2539329,1,2,8
1,1,2398795,2,3,7


In [9]:
products = pd.read_csv('products.csv')
print(products.shape)
products.head()


(49688, 4)


,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13


In [10]:
frequent_name=['Banana','Bag of Organic Bananas','Organic Strawberries','Organic Baby Spinach','Organic Hass Avocado','Organic Avocado',
'Large Lemon','Strawberries','Limes','Organic Whole Milk']
condition= products['product_name'].isin(frequent_name)
columns=['product_id','product_name']
products_subset=products.loc[condition,columns]
products_subset.head()

,product_id,product_name
13175,13176,Bag of Organic Bananas
16796,16797,Strawberries
21136,21137,Organic Strawberries
21902,21903,Organic Baby Spinach
24851,24852,Banana


In [11]:
order_products=pd.concat([order_products__prior,order_products__train])
order_products.head()

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


In [12]:



columns = ['order_id', 'product_id', 'add_to_cart_order']

order_products_subset = order_products[columns]
order_products_subset.head(11)

,order_id,product_id,add_to_cart_order
0,2,33120,1
1,2,28985,2
2,2,9327,3
3,2,45918,4
4,2,30035,5
5,2,17794,6
6,2,40141,7
7,2,1819,8
8,2,43668,9
9,3,33754,1


In [13]:
merged=pd.merge(order_products_subset,products_subset,how='inner',on='product_id')
merged.head()

,order_id,product_id,add_to_cart_order,product_name
0,3,21903,4,Organic Baby Spinach
1,26,21903,6,Organic Baby Spinach
2,31,21903,3,Organic Baby Spinach
3,39,21903,4,Organic Baby Spinach
4,56,21903,8,Organic Baby Spinach


In [14]:
final = pd.merge(merged,
                 orders[['order_number','order_id','order_dow']],
                 how='inner',
                 on='order_id'
                 )
final.head(20)


,order_id,product_id,add_to_cart_order,product_name,order_number,order_dow
0,3,21903,4,Organic Baby Spinach,16,5
1,26,21903,6,Organic Baby Spinach,2,0
2,26,24852,2,Banana,2,0
3,26,47766,8,Organic Avocado,2,0
4,31,21903,3,Organic Baby Spinach,7,6
5,39,21903,4,Organic Baby Spinach,8,5
6,56,21903,8,Organic Baby Spinach,8,0
7,56,47209,7,Organic Hass Avocado,8,0
8,57,21903,1,Organic Baby Spinach,1,6
9,57,47209,6,Organic Hass Avocado,1,6


In [15]:
final.sort_values(by=['product_name'])

,order_id,product_id,add_to_cart_order,product_name,order_number,order_dow
688948,958798,13176,5,Bag of Organic Bananas,63,4
1075649,2155903,13176,1,Bag of Organic Bananas,72,5
1075650,2156132,13176,1,Bag of Organic Bananas,10,5
1075651,2156249,13176,5,Bag of Organic Bananas,11,1
1075652,2156270,13176,2,Bag of Organic Bananas,44,5
...,...,...,...,...,...,...
2306890,484,16797,3,Strawberries,16,3
2306891,498,16797,2,Strawberries,10,1
2306892,541,16797,2,Strawberries,12,2
1078776,2660230,16797,9,Strawberries,51,6


In [16]:
final['product_name'].value_counts()


Banana                    491291
Bag of Organic Bananas    394930
Organic Strawberries      275577
Organic Baby Spinach      251705
Organic Hass Avocado      220877
Organic Avocado           184224
Large Lemon               160792
Strawberries              149445
Limes                     146660
Organic Whole Milk        142813
Name: product_name, dtype: int64

## Reshape Data Section

- Replicate the lesson code
- Complete the code cells we skipped near the beginning of the notebook
- Table 2 --> Tidy
- Tidy --> Table 2
- Load seaborn's `flights` dataset by running the cell below. Then create a pivot table showing the number of passengers by month and year. Use year for the index and month for the columns. You've done it right if you get 112 passengers for January 1949 and 432 passengers for December 1960.

In [0]:
%matplotlib inline
import pandas as pd
import numpy as np
import seaborn as sns

table1 = pd.DataFrame(
    [[np.nan, 2],
     [16,    11], 
     [3,      1]],
    index=['John Smith', 'Jane Doe', 'Mary Johnson'], 
    columns=['treatmenta', 'treatmentb'])

table2 = pd.DataFrame([[np.nan,16.0,3.0],
                       [2.0,11.0,1.0]],
                      index=['treatmenta','treatmentb'],
                      columns=['John Smith', 'Jane Doe', 'Mary Johnson'])

In [18]:
##### YOUR CODE HERE #####
table2 = table2.reset_index()
table2

,index,John Smith,Jane Doe,Mary Johnson
0,treatmenta,NaN,16.0,3.0
1,treatmentb,2.0,11.0,1.0


In [19]:
tidy2 = table2.melt(id_vars='index',value_vars=['John Smith', 'Jane Doe', 'Mary Johnson'])
tidy2

,index,variable,value
0,treatmenta,John Smith,NaN
1,treatmentb,John Smith,2.0
2,treatmenta,Jane Doe,16.0
3,treatmentb,Jane Doe,11.0
4,treatmenta,Mary Johnson,3.0
5,treatmentb,Mary Johnson,1.0


In [20]:
tidy2=tidy2.rename(columns={
    'index': 'trt',
    'variable': 'name',
    'value': 'result'
})
tidy2

,trt,name,result
0,treatmenta,John Smith,NaN
1,treatmentb,John Smith,2.0
2,treatmenta,Jane Doe,16.0
3,treatmentb,Jane Doe,11.0
4,treatmenta,Mary Johnson,3.0
5,treatmentb,Mary Johnson,1.0


In [21]:
tidy2.trt = tidy2.trt.str.replace('treatment','')
tidy2

,trt,name,result
0,a,John Smith,NaN
1,b,John Smith,2.0
2,a,Jane Doe,16.0
3,b,Jane Doe,11.0
4,a,Mary Johnson,3.0
5,b,Mary Johnson,1.0


In [22]:
wide=tidy2.pivot_table(index='trt',columns='name',values='result')
wide

name,Jane Doe,John Smith,Mary Johnson
trt,,,
a,16.0,NaN,3.0
b,11.0,2.0,1.0


In [23]:
import seaborn as sns


flights = sns.load_dataset('flights')
flights.pivot_table(index='year',
                    columns='month',
                    values=flights)

passengers                       ...                                    
month    January February March April  ... September October November December
year                                   ...                                    
1949         112      118   132   129  ...       136     119      104      118
1950         115      126   141   135  ...       158     133      114      140
1951         145      150   178   163  ...       184     162      146      166
1952         171      180   193   181  ...       209     191      172      194
1953         196      196   236   235  ...       237     211      180      201
1954         204      188   235   227  ...       259     229      203      229
1955         242      233   267   269  ...       312     274      237      278
1956         284      277   317   313  ...       355     306      271      306
1957         315      301   356   348  ...       404     347      305      336
1958         340      318   362   348  ...       404     359      310      337
1959         360      342   406   396  ...       463     407      362      405
1960         417      391   419   461  ...       508     461      390      432

[12 rows x 12 columns]

## Join Data Stretch Challenge

The [Instacart blog post](https://tech.instacart.com/3-million-instacart-orders-open-sourced-d40d29ead6f2) has a visualization of "**Popular products** purchased earliest in the day (green) and latest in the day (red)." 

The post says,

> "We can also see the time of day that users purchase specific products.

> Healthier snacks and staples tend to be purchased earlier in the day, whereas ice cream (especially Half Baked and The Tonight Dough) are far more popular when customers are ordering in the evening.

> **In fact, of the top 25 latest ordered products, the first 24 are ice cream! The last one, of course, is a frozen pizza.**"

Your challenge is to reproduce the list of the top 25 latest ordered popular products.

We'll define "popular products" as products with more than 2,900 orders.



In [36]:
##### YOUR CODE HERE #####

columns = ['order_id', 
           'order_number', 
           'order_hour_of_day']
print(max(orders['order_hour_of_day']))
condition= orders['order_hour_of_day']==23

orders_subset = orders.loc[condition,columns]

orders_subset.head()





23


,order_id,order_number,order_hour_of_day
182,3069802,23,23
474,1210985,5,23
811,51418,29,23
833,1102893,51,23
842,3370941,60,23


In [30]:

columns = ['order_id', 'product_id', 'add_to_cart_order']

order_products_subset = order_products[columns]
order_products_subset.head(11)


,order_id,product_id,add_to_cart_order
0,2,33120,1
1,2,28985,2
2,2,9327,3
3,2,45918,4
4,2,30035,5
5,2,17794,6
6,2,40141,7
7,2,1819,8
8,2,43668,9
9,3,33754,1


In [40]:
merged = pd.merge(orders_subset, order_products_subset, how='inner', on='order_id')
merged.head(11)
final = pd.merge(merged, products[['product_id', 'product_name']], how='left', on='product_id')
final.head(500)
final['product_name'].value_counts()[:25]

Banana                                  5587
Bag of Organic Bananas                  5098
Organic Strawberries                    3677
Organic Baby Spinach                    3304
Organic Hass Avocado                    2702
Organic Avocado                         2280
Strawberries                            1920
Large Lemon                             1906
Organic Raspberries                     1872
Organic Whole Milk                      1870
Limes                                   1823
Organic Blueberries                     1533
Organic Zucchini                        1508
Organic Garlic                          1463
Organic Yellow Onion                    1394
Cucumber Kirby                          1250
Organic Lemon                           1191
Organic Cucumber                        1180
Organic Grape Tomatoes                  1165
Seedless Red Grapes                     1137
Organic Baby Carrots                    1103
Organic Cilantro                        1074
Organic La

## Reshape Data Stretch Challenge

_Try whatever sounds most interesting to you!_

- Replicate more of Instacart's visualization showing "Hour of Day Ordered" vs "Percent of Orders by Product"
- Replicate parts of the other visualization from [Instacart's blog post](https://tech.instacart.com/3-million-instacart-orders-open-sourced-d40d29ead6f2), showing "Number of Purchases" vs "Percent Reorder Purchases"
- Get the most recent order for each user in Instacart's dataset. This is a useful baseline when [predicting a user's next order](https://www.kaggle.com/c/instacart-market-basket-analysis)
- Replicate parts of the blog post linked at the top of this notebook: [Modern Pandas, Part 5: Tidy Data](https://tomaugspurger.github.io/modern-5-tidy.html)

In [0]:
##### YOUR CODE HERE #####